<a href="https://colab.research.google.com/github/shunichi922/TEST/blob/TEST/4_4_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.4 scikit-learn

## 4.4.1 前処理

### ●欠損値への対応

In [ ]:
import numpy as np
import pandas as pd
# サンプルのデータセットを作成
df = pd.DataFrame(
    {
        "A": [1, np.nan, 3, 4, 5],
        "B": [6, 7, 8, np.nan, 10],
        "C": [11, 12, 13, 14, 15]
    }
)
df

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


In [ ]:
# 各要素が欠損値かどうか確かめる
df.isnull()

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


#### ■写経中に遭遇したscikit-learnのワーニングに対応してみた
https://qiita.com/y_nishimura/items/6d022a25beb21a186d60

In [ ]:
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

# 平均値で欠損値を補完するインスタンスを作成する
#imp = Imputer(strategy="mean", axis=0)
imp = SimpleImputer(strategy="mean")

# 欠損値を補完
imp.fit(df)
imp.transform(df)

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

### ●カテゴリ変数のエンコーディング

In [ ]:
import pandas as pd
df = pd.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "B": ["a", "b", "a", "b", "c"]
    }
)
df

,A,B
0,1,a
1,2,b
2,3,a
3,4,b
4,5,c


In [ ]:
from sklearn.preprocessing import LabelEncoder
# ラベルエンコーダのインスタンスを生成
le = LabelEncoder()
# ラベルのエンコーディング
le.fit(df["B"])
le.transform(df["B"])

array([0, 1, 0, 1, 2])

In [ ]:
# 元の値を確認
le.classes_

array(['a', 'b', 'c'], dtype=object)

#### ■写経中に遭遇したscikit-learnのワーニングに対応してみた
https://qiita.com/y_nishimura/items/6d022a25beb21a186d60

In [ ]:
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

# DataFrameをコピー
df_ohe = df.copy()

# ラベルエンコーダのインスタンス化
#le = LabelEncoder()

# 英語のa、b、cを1、2、3に変換
#df_ohe["B"] = le.fit_transform(df_ohe["B"])

# One-hotエンコーダのインスタンス化
#ohe = OneHotEncoder(categorical_features=[1])

# ColumnTransformerのインスタンス化
ct = ColumnTransformer([('B_x', CountVectorizer(analyzer=lambda x: [x]), 'B')],
    remainder = 'passthrough')

# One-hotエンコーディング
#ohe.fit_transform(df_ohe).toarray()
ct.fit_transform(df_ohe)

array([[1, 0, 0, 1],
       [0, 1, 0, 2],
       [1, 0, 0, 3],
       [0, 1, 0, 4],
       [0, 0, 1, 5]])

### ●特徴量の正規化